In [ ]:
%pip install distance

In [ ]:
%pip install fuzzywuzzy

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings, nltk, re, distance
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tqdm.notebook import tqdm
tqdm.pandas()
from fuzzywuzzy import fuzz
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout   
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
import scikitplot as skplt
import joblib
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, LSTM
from bs4 import BeautifulSoup
from tqdm import tqdm



In [ ]:
%pip install tqdm

In [ ]:
nltk.download('stopwords')

In [ ]:
#Load the Training Data
train = pd.read_csv('train.csv')
train = train.sample(n=100000).reset_index(drop=True)
train.head()

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
#Dropping Null/NA values
train = train.dropna()

In [ ]:
train.duplicated().sum()

In [ ]:
#Loading Test Data
test = pd.read_csv('test.csv')
test = test.sample(n=100000).reset_index(drop=True)
test.head()

In [ ]:
test.shape

In [ ]:
test.isnull().sum()

In [ ]:
test = train.dropna()

In [ ]:
test.duplicated().sum()

In [ ]:
#function to clean and standardize the text data
def processText(question):
    
    # Converts the input text to lowercase and removes any leading or trailing whitespace.
    question = str(question).lower().strip()

    # Replacements for Special Characters and Symbols.
    question = question.replace('%',' percent ')
    question = question.replace('$',' dollar ')
    question = question.replace('£',' pound ')
    question = question.replace('₹', ' rupee ')
    question = question.replace('€', ' euro ')
    question = question.replace('@', ' at ')
    question = question.replace('[math]','')
    question = question.replace(',000,000,000 ', 'b ')
    question = question.replace(',000,000 ', 'm ')
    question = question.replace(',000 ', 'k ')
    
    # Numeric abbreviations.
    question = re.sub(r'([0-9]+)000000000', r'\1b', question)
    question = re.sub(r'([0-9]+)000000', r'\1m', question)
    question = re.sub(r'([0-9]+)000', r'\1k', question)


    # Contraction Expansions.
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }


    decontracted_question = []

    for word in question.split():
      if word in contractions:
        word = contractions[word]
      decontracted_question.append(word)
    
    question = ' '.join(decontracted_question)
    question = question.replace("'ve", " have")
    question = question.replace("n't", " not")
    question = question.replace("'re", " are")
    question = question.replace("'ll", " will")
    question = re.sub(r"[^A-Za-z0-9]", " ", question)
    question = re.sub('[,\.\(\)?"\']'," ",question)
    question = re.sub(r"what's", "", question)
    question = re.sub(r"What's", "", question)
    question = re.sub(r"\'s", " ", question)
    question = re.sub(r"\'ve", " have ", question)
    question = re.sub(r"can't", "cannot ", question)
    question = re.sub(r"n't", " not ", question)
    question = re.sub(r"I'm", "I am", question)
    question = re.sub(r" m ", " am ", question)
    question = re.sub(r"\'re", " are ", question)
    question = re.sub(r"\'d", " would ", question)
    question = re.sub(r"\'ll", " will ", question)
    question = re.sub(r"\0k ", "0000 ", question)
    question = re.sub(r" e g ", " eg ", question)
    question = re.sub(r" b g ", " bg ", question)
    question = re.sub(r"\0s", "0", question)
    question = re.sub(r" 9 11 ", "911", question)
    question = re.sub(r"e-mail", "email", question)
    question = re.sub(r"\s{2,}", " ", question)
    question = re.sub(r"quikly", "quickly", question)
    question = re.sub(r" usa ", " America ", question)
    question = re.sub(r" USA ", " America ", question)
    question = re.sub(r" u s ", " America ", question)
    question = re.sub(r" uk ", " England ", question)
    question = re.sub(r" UK ", " England ", question)
    question = re.sub(r"india", "India", question)
    question = re.sub(r"china", "China", question)
    question = re.sub(r"chinese", "Chinese", question) 
    question = re.sub(r"imrovement", "improvement", question)
    question = re.sub(r"intially", "initially", question)
    question = re.sub(r"quora", "Quora", question)
    question = re.sub(r" dms ", "direct messages ", question)  
    question = re.sub(r"demonitization", "demonetization", question) 
    question = re.sub(r"actived", "active", question)
    question = re.sub(r"kms", " kilometers ", question)
    question = re.sub(r"KMs", " kilometers ", question)
    question = re.sub(r" cs ", " computer science ", question) 
    question = re.sub(r" upvotes ", " up votes ", question)
    question = re.sub(r" iPhone ", " phone ", question)
    question = re.sub(r"\0rs ", " rs ", question) 
    question = re.sub(r"calender", "calendar", question)
    question = re.sub(r"ios", "operating system", question)
    question = re.sub(r"gps", "GPS", question)
    question = re.sub(r"gst", "GST", question)
    question = re.sub(r"programing", "programming", question)
    question = re.sub(r"bestfriend", "best friend", question)
    question = re.sub(r"dna", "DNA", question)
    question = re.sub(r"III", "3", question) 
    question = re.sub(r"the US", "America", question)
    question = re.sub(r"Astrology", "astrology", question)
    question = re.sub(r"Method", "method", question)
    question = re.sub(r"Find", "find", question) 
    question = re.sub(r"banglore", "Banglore", question)
    question = re.sub(r" J K ", " JK ", question)

    # Eliminate HTML tags
    question = BeautifulSoup(question)
    question = question.get_text()

    # Remove punctuation characters
    pattern = re.compile('\W')
    question = re.sub(pattern,' ',question).strip()
    
    return question


In [ ]:
#The progress_apply() method is used to apply a function to each element in a Series (column) of a DataFrame while providing a progress bar to track the progress of the operation.(Refer From Stackoverflow)

tqdm.pandas()  # Enable progress bar for pandas apply
train.question1 = train.question1.progress_apply(processText)
train.question2 = train.question2.progress_apply(processText)

In [ ]:
#The progress_apply() method is used to apply a function to each element in a Series (column) of a DataFrame while providing a progress bar to track the progress of the operation.

tqdm.pandas()  # Enable progress bar for pandas apply
test.question1 = test.question1.progress_apply(processText)
test.question2 = test.question2.progress_apply(processText)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Calculate the length of each text entry
train['q1_len'] = train.question1.apply(len)
train['q2_len'] = train.question2.apply(len)

In [ ]:
# Finding number of words in each question
train['q1_num_words'] = train.question1.apply(lambda sent: len(sent.split()))
train['q2_num_words'] = train.question2.apply(lambda sent: len(sent.split()))

In [ ]:
# This function takes a row of data as input and finds the number of common words
# between two questions in that row.

def find_common_words(row):
    # Convert the words in 'question1' to lowercase and create a set to store them.
    words_set1 = set(map(lambda word: word.lower().strip(), row['question1'].split()))
    
    # Convert the words in 'question2' to lowercase and create a set to store them.
    words_set2 = set(map(lambda word: word.lower().strip(), row['question2'].split()))
    
    # Calculate the number of common words between the two sets.
    common_word_count = len(words_set1 & words_set2)
    
    # Return the count of common words.
    return common_word_count

In [ ]:
# Assuming 'train' is a DataFrame, this line of code applies the 'find_common_words' function
# to each row in the DataFrame and creates a new column 'common_words' to store the results.
train['common_words'] = train.apply(find_common_words, axis=1)

In [ ]:
train.head()

In [ ]:
test['q1_len'] = test.question1.apply(len)
test['q2_len'] = test.question2.apply(len)

In [ ]:
test['q1_num_words'] = test.question1.apply(lambda sent: len(sent.split()))
test['q2_num_words'] = test.question2.apply(lambda sent: len(sent.split()))

In [ ]:
test['common_words'] = test.apply(find_common_words,axis=1)

In [ ]:
test.head()

In [ ]:
# This function calculates the total number of words in two questions of a row in a DataFrame.

def calculate_total_words(row):
    # Convert the words in 'question1' to lowercase and create a set to store them.
    words_set1 = set(map(lambda word: word.lower().strip(), row['question1'].split()))
    
    # Convert the words in 'question2' to lowercase and create a set to store them.
    words_set2 = set(map(lambda word: word.lower().strip(), row['question2'].split()))
    
    # Calculate the total number of words by adding the lengths of the two sets.
    total_word_count = len(words_set1) + len(words_set2)
    
    # Return the total word count.
    return total_word_count

In [ ]:
# Applying the 'calculate_total_words' function to each row in the DataFrame and creates a new column 'total_words' to store the results.
train['total_words'] = train.apply(calculate_total_words, axis=1)

# Display the first few rows of the DataFrame to show the results.
train.head()

In [ ]:
 #The 'shared_words' column quantifies the similarity between pairs of questions by calculating the ratio of shared words to the total number of words in those questions. 
 #This measure can help in understanding the textual similarity between different pairs of questions and can potentially be used as a feature for further analysis or modeling purposes.

train['shared_words'] = round(train.common_words / train.total_words,2)
train.head()

In [ ]:
test['total_words'] = test.apply(calculate_total_words,axis=1)
test.head()

In [ ]:
test['shared_words'] = round(test.common_words / test.total_words,2)
test.head()

In [ ]:
# This function computes token-based features for two input questions in a row.

def compute_token_similarity_features(row):
    # Extract the text of 'question1' and 'question2'.
    ques1 = row['question1']
    ques2 = row['question2']
    
    # A small constant to avoid division by zero.
    epsilon = 0.0001
    
    # Initialize an array to store the token-based features.
    token_features = np.zeros((8,), 'float32')
    
    # Calculate the lengths of 'question1' and 'question2'.
    q1_len, q2_len = len(ques1), len(ques2)
    
    # Split the questions into tokens.
    tokens_ques1 = ques1.split()
    tokens_ques2 = ques2.split()
    
    # Check if either of the questions has no tokens.
    if len(tokens_ques1) == 0 or len(tokens_ques2) == 0:
        return token_features
    
    # Create sets of non-stopwords and stopwords for both questions.
    non_stopwords_q1 = set([word for word in tokens_ques1 if word not in stopwords.words('english')])
    non_stopwords_q2 = set([word for word in tokens_ques2 if word not in stopwords.words('english')])

    stopwords_q1 = set([word for word in tokens_ques1 if word in stopwords.words('english')])
    stopwords_q2 = set([word for word in tokens_ques2 if word in stopwords.words('english')])
    
    # Calculate the counts of common non-stopwords, stopwords, and tokens.
    common_word_count = len(non_stopwords_q1.intersection(non_stopwords_q2))
    common_stop_count = len(stopwords_q1.intersection(stopwords_q2))
    common_token_count = len(set(tokens_ques1).intersection(set(tokens_ques2)))
    
    # Calculate the token-based features.
    token_features[0] = common_word_count / (min(q1_len, q2_len) + epsilon)
    token_features[1] = common_word_count / (max(q1_len, q2_len) + epsilon)
    token_features[2] = common_stop_count / (min(q1_len, q2_len) + epsilon)
    token_features[3] = common_stop_count / (max(q1_len, q2_len) + epsilon)
    token_features[4] = common_token_count / (min(q1_len, q2_len) + epsilon)
    token_features[5] = common_token_count / (max(q1_len, q2_len) + epsilon)
    token_features[6] = int(tokens_ques1[0] == tokens_ques2[0])
    token_features[7] = int(tokens_ques1[-1] == tokens_ques2[-1])
    
    return token_features


In [ ]:
#Referred from Kaggle Notebook.
token_features = train.progress_apply(compute_token_similarity_features,axis=1)

train['cwc_min'] = list(map(lambda x: x[0], token_features))
train['cwc_max'] = list(map(lambda x: x[1], token_features))
train['csc_min'] = list(map(lambda x: x[2], token_features))
train['csc_max'] = list(map(lambda x: x[3], token_features))
train['ctc_min'] = list(map(lambda x: x[4], token_features))
train['ctc_max'] = list(map(lambda x: x[5], token_features))
train['first_word_same'] = list(map(lambda x: x[6], token_features))
train['last_word_same'] = list(map(lambda x: x[7], token_features))

train.head()

In [ ]:
token_features = test.progress_apply(compute_token_similarity_features,axis=1)

test['cwc_min'] = list(map(lambda x: x[0], token_features))
test['cwc_max'] = list(map(lambda x: x[1], token_features))
test['csc_min'] = list(map(lambda x: x[2], token_features))
test['csc_max'] = list(map(lambda x: x[3], token_features))
test['ctc_min'] = list(map(lambda x: x[4], token_features))
test['ctc_max'] = list(map(lambda x: x[5], token_features))
test['first_word_same'] = list(map(lambda x: x[6], token_features))
test['last_word_same'] = list(map(lambda x: x[7], token_features))

test.head()

In [ ]:
# This function computes length-based features for two input questions in a row.

def compute_length_similarity_features(row):   
    # Extract the text of 'question1' and 'question2'.
    ques1 = row['question1']
    ques2 = row['question2']  
    
    # Initialize an array to store the length-based features.
    length_features = np.zeros((3,), 'float32')
    
    # Split the questions into tokens.
    q1_tokens = ques1.split()
    q2_tokens = ques2.split()
    
    # Check if either of the questions has no tokens.
    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features

    # Calculate the absolute difference in token count between the two questions.
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))
    
    # Calculate the average token count between the two questions.
    length_features[1] = (len(q1_tokens) + len(q2_tokens)) / 2
    
    # Find the common substrings between the two questions and normalize by the length of the shorter question.
    strs = list(distance.lcsubstrings(ques1, ques2))
    length_features[2] = len(strs) / (min(len(ques1), len(ques2)) + 1) 
    
    return length_features


In [ ]:
# Calculate length-based features for each row in the 'train' DataFrame.
length_features = train.progress_apply(compute_length_similarity_features, axis=1)

# Extract and assign specific length-based features to new columns.
train['mean_question_length'] = list(map(lambda x: x[0], length_features))
train['absolute_length_difference'] = list(map(lambda x: x[1], length_features))
train['longest_common_substring_ratio'] = list(map(lambda x: x[2], length_features))

# Display the updated DataFrame with the new length-based features.
train.head()

In [ ]:
length_features_test = test.progress_apply(compute_length_similarity_features,axis=1)

test['mean_question_length'] = list(map(lambda x: x[0], length_features_test))
test['absolute_length_difference'] = list(map(lambda x: x[1], length_features_test))
test['longest_common_substring_ratio'] = list(map(lambda x: x[2], length_features_test))

test.head()

In [ ]:
# This function calculates fuzzy matching features for two input questions in a row.

def compute_fuzzy_matching_features(row):
    # Extract the text of 'question1' and 'question2'.
    ques1 = row['question1']
    ques2 = row['question2']

    # Initialize an array to store the fuzzy matching features.
    fuzzy_features = np.zeros((4,), 'float32')

    # Calculate the fuzzy ratio between the two questions.
    fuzzy_features[0] = fuzz.QRatio(ques1, ques2)

    # Calculate the partial fuzzy ratio between the two questions.
    fuzzy_features[1] = fuzz.partial_ratio(ques1, ques2)

    # Calculate the token set ratio between the two questions.
    fuzzy_features[2] = fuzz.token_set_ratio(ques1, ques2)

    # Calculate the token sort ratio between the two questions.
    fuzzy_features[3] = fuzz.token_sort_ratio(ques1, ques2)

    return fuzzy_features


In [ ]:
# Calculate fuzzy matching features for each row in the 'train' DataFrame.
fuzzy_features = train.progress_apply(compute_fuzzy_matching_features, axis=1)

# Extract and assign specific fuzzy matching features to new columns.
train['fuzzy_ratio'] = list(map(lambda x: x[0], fuzzy_features))
train['fuzzy_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
train['token_set_ratio'] = list(map(lambda x: x[2], fuzzy_features))
train['token_sort_ratio'] = list(map(lambda x: x[3], fuzzy_features))

# Display the updated DataFrame with the new fuzzy matching features.
train.head()


In [ ]:
fuzz_features_test = test.progress_apply(compute_fuzzy_matching_features,axis=1)

test['fuzzy_ratio'] = list(map(lambda x: x[0],fuzz_features_test))
test['fuzzy_partial_ratio'] = list(map(lambda x: x[1],fuzz_features_test))
test['token_set_ratio'] = list(map(lambda x: x[2],fuzz_features_test))
test['token_sort_ratio'] = list(map(lambda x: x[3],fuzz_features_test))

test.head()

In [ ]:
#cwc_min: Feature representing the minimum common word count between two questions.
#cwc_max: Feature representing the maximum common word count between two questions.
#ctc_min: Feature representing the minimum common token count between two questions.
#ctc_max: Feature representing the maximum common token count between two questions.
#csc_min: Feature representing the minimum common stop word count between two questions.
#csc_max: Feature representing the maximum common stop word count between two questions.
#is_duplicate: Target variable indicating whether the question pair is a duplicate (1) or not (0).
#hue='is_duplicate': This parameter specifies that the 'is_duplicate' column will be used to color-code the data points in the pair plot.
#In other words, it allows us to differentiate between duplicate and non-duplicate question pairs based on colors.

#The scatter plots can show how similar or dissimilar feature values are for duplicate and non-duplicate pairs. 
#Clusters or patterns in the scatter plots can indicate the discriminatory power of features in classifying question pairs.
#The histograms show the distribution of individual features, giving insight into their ranges and variations.
#The diagonal axis shows the kernel density estimate, which gives a smoothed representation of the distribution of each feature.

sns.pairplot(train[['cwc_min','cwc_max','ctc_min','ctc_max','csc_min','csc_max','is_duplicate']], hue='is_duplicate')

In [ ]:
#first_word_same: A feature representing whether the first word of both questions in a pair is the same (1) or not (0).
#last_word_same: A feature representing whether the last word of both questions in a pair is the same (1) or not (0).
#is_duplicate: The target variable indicating whether the question pair is a duplicate (1) or not (0).
#hue='is_duplicate': This parameter assigns the 'is_duplicate' column as the basis for color-coding the data points in the pair plot. 
#By using this parameter, the code allows you to visually distinguish between duplicate and non-duplicate question pairs using different colors.

#The scatter plots will indicate how the values of 'first_word_same' and 'last_word_same' vary for both duplicate and non-duplicate question pairs.
#Clusters or trends in these scatter plots can provide insights into how these features contribute to the classification.
#The histograms will illustrate the distribution of each feature, showing the frequency and spread of values.

sns.pairplot(train[['first_word_same','last_word_same','is_duplicate']],hue='is_duplicate')

In [ ]:
#train[['mean_ques_len', 'abs_len_diff', 'longest_sub_ratio', 'is_duplicate']]: This section of the code selects specific columns from the 'train' DataFrame.
#The columns selected are as follows:
#mean_ques_len: A feature representing the average length of tokens in both questions in a pair.
#abs_len_diff: A feature representing the absolute difference in the number of tokens between the two questions in a pair.
#longest_sub_ratio: A feature representing the ratio of the length of the longest common substring to the minimum of the lengths of the two questions in a pair.
#is_duplicate: The target variable indicating whether the question pair is a duplicate (1) or not (0).
#hue='is_duplicate': This parameter assigns the 'is_duplicate' column as the basis for color-coding the data points in the pair plot. By using this parameter, the code allows you to visually distinguish between duplicate and non-duplicate question pairs using different colors.

sns.pairplot(train[['mean_question_length','absolute_length_difference','longest_common_substring_ratio','is_duplicate']],hue='is_duplicate')

In [ ]:
#train[['fuzz_ratio', 'fuzz_partial_ratio', 'token_set_ratio', 'token_sort_ratio', 'is_duplicate']]: This part of the code selects specific columns from the 'train' DataFrame. The columns selected are as follows:

#fuzz_ratio: A feature representing the fuzz ratio, which measures the similarity between two strings based on different matching criteria.
#fuzz_partial_ratio: A feature representing the fuzz partial ratio, which focuses on partial matching between two strings.
#token_set_ratio: A feature representing the token set ratio, which measures the similarity between two strings based on their token sets.
#token_sort_ratio: A feature representing the token sort ratio, which measures the similarity between two strings based on sorted tokens.
#is_duplicate: The target variable indicating whether the question pair is a duplicate (1) or not (0).
#hue='is_duplicate': This parameter assigns the 'is_duplicate' column as the basis for color-coding the data points in the pair plot. This color coding allows you to visually differentiate between duplicate and non-duplicate question pairs.


#Scatter plots visually represent the relationship between feature values and the 'is_duplicate' label for both duplicate and non-duplicate question pairs.
#Histograms illustrate the distribution of each feature, providing insights into their values' frequency and spread.

sns.pairplot(train[['fuzzy_ratio','fuzzy_partial_ratio','token_set_ratio','token_sort_ratio','is_duplicate']],hue='is_duplicate')

In [ ]:

#The MinMaxScaler is a popular technique for transforming features to a specific range, typically [0, 1].

#scaled_train = scaler.fit_transform(...): This line applies the MinMaxScaler to a subset of features in the 'train' dataset. The features being scaled are:
#cwc_min: Minimum common word count
#cwc_max: Maximum common word count
#ctc_min: Minimum common token count
#ctc_max: Maximum common token count
#csc_min: Minimum common stop word count
#csc_max: Maximum common stop word count
#first_word_same: Whether the first word is the same
#last_word_same: Whether the last word is the same
#abs_len_diff: Absolute length difference
#mean_ques_len: Mean question length
#longest_sub_ratio: Longest common substring ratio
#fuzz_ratio: Fuzzy string matching ratio
#fuzz_partial_ratio: Fuzzy partial string matching ratio
#token_set_ratio: Token set matching ratio
#token_sort_ratio: Token sort matching ratio
#The .fit_transform() method of the scaler scales the selected features and returns a new array scaled_train containing the scaled values.

#target_train = train.is_duplicate.values: In this line, the target variable 'is_duplicate' is extracted from the 'train' dataset and converted into a NumPy array.
#This prepares the target variable for further analysis, such as model training and evaluation.

#Overall, this showcases the application of feature scaling using the MinMaxScaler to specific features in the 'train' dataset. 
#Feature scaling is important for ensuring that the features are on a similar scale, which can improve the performance of machine learning algorithms that are sensitive to feature magnitudes. 
#The scaled features, stored in scaled_train, and the target variable, stored in target_train, are now ready to be used for building and training machine learning models.


mm_scaler = MinMaxScaler()
scaled_train = mm_scaler.fit_transform(train[['cwc_min','cwc_max','ctc_min','ctc_max','csc_min','csc_max','first_word_same','last_word_same','absolute_length_difference','mean_question_length','longest_common_substring_ratio','fuzzy_ratio','fuzzy_partial_ratio','token_set_ratio','token_sort_ratio']])
target_train = train.is_duplicate.values

In [ ]:

scaled_test = mm_scaler.fit_transform(test[['cwc_min','cwc_max','ctc_min','ctc_max','csc_min','csc_max','first_word_same','last_word_same','absolute_length_difference','mean_question_length','longest_common_substring_ratio','fuzzy_ratio','fuzzy_partial_ratio','token_set_ratio','token_sort_ratio']])
target_test = test.is_duplicate.values

In [ ]:
#An instance of the PCA class is created. The PCA class from the scikit-learn library is used for performing Principal Component Analysis. The parameters are as follows:

#n_components=2: This parameter specifies the number of principal components to retain after the transformation. In this case, the data will be reduced to a 2-dimensional space.
#random_state=101: This parameter sets the random seed for reproducibility. The same random seed ensures that the results will be consistent across different runs.
#X_train_pca = pca.fit_transform(scaled_train): This line applies PCA transformation to the scaled features of the 'train' dataset (scaled_train). 
#The .fit_transform() method of the PCA class computes the principal components and performs the dimensionality reduction. The result is stored in the variable X_train_pca.
#X_train_pca: This new variable holds the transformed data after applying PCA. The data is now represented in a lower-dimensional space, with each row representing a data point and each column corresponding to a principal component.
#PCA works by finding orthogonal axes (principal components) along which the variance of the data is maximized. 
#The first principal component accounts for the most variance, the second for the second most, and so on. 
#By choosing only a few principal components (in this case, 2), the code reduces the data's dimensionality while retaining the most important information.
#The resulting X_train_pca can be used for various purposes, such as visualization, clustering, or as input features for machine learning models.
#By reducing the data to only two dimensions, you can create scatter plots or visualizations to gain insights into the structure and patterns within the data.

pca = PCA(n_components=2,random_state=101)
X_train_pca = pca.fit_transform(scaled_train)
X_test_pca = pca.fit_transform(scaled_test)

In [ ]:
import matplotlib.pyplot as plt

# Set the figure size for the scatter plot.
plt.figure(figsize=(12, 8))

# Create a scatter plot using the first two principal components from X_train_pca.
# 'X_train_pca[:, 0]' represents the first principal component, and 'X_train_pca[:, 1]'
# represents the second principal component. 'c=target_train' colors the points based on
# their corresponding target labels.

fig = plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=target_train)

# Add a legend to the plot. 'fig.legend_elements()[0]' extracts legend handles and
# 'labels=list([0,1])' specifies the labels for the legend.

plt.legend(handles=fig.legend_elements()[0], labels=list([0, 1]))

# Display the scatter plot.
plt.show()


In [ ]:
plt.figure(figsize=(12,8))
fig = plt.scatter(X_test_pca[:,0],X_train_pca[:,1],c=target_test)
plt.legend(handles=fig.legend_elements()[0],labels=list([0,1]))
plt.show();

In [ ]:
rest_train_data = train.drop(['id','qid1','qid2','question1','question2'],axis=1)
rest_train_data.head()

In [ ]:
rest_test_data = test.drop(['id','qid1','qid2','question1','question2'],axis=1)
rest_test_data.head()

In [ ]:
ques_train_df = train[['question1','question2']]
ques_train_df.head()

In [ ]:
ques_test_df = test[['question1','question2']]
ques_test_df.head()

In [ ]:
# Combine all questions from 'ques_train_df' into a single array.
questions_train = np.array(list(ques_train_df.question1) + list(ques_train_df.question2))

# Create a CountVectorizer with a maximum of 500 features (words).
cv = CountVectorizer(max_features=500)

# Transform and convert the text data from 'questions_train' into a numerical format.
# The 'toarray()' method converts the result into a numerical array.
# Split the transformed data into two arrays, one for 'question1' and one for 'question2'.
ques1_train_arr, ques2_train_arr = np.vsplit(cv.fit_transform(questions_train).toarray(), 2)


In [ ]:
questions_test = np.array(list(ques_test_df.question1) + list(ques_test_df.question2))
cv = CountVectorizer(max_features=500)
ques1_test_arr, ques2_test_arr = np.vsplit(cv.fit_transform(questions_test).toarray(),2)

In [ ]:

# Create a DataFrame 'question1_df' from the numerical representations of 'question1'.
question1_df = pd.DataFrame(ques1_train_arr)

# Create a DataFrame 'question2_df' from the numerical representations of 'question2'.
question2_df = pd.DataFrame(ques2_train_arr)

# Concatenate the two DataFrames 'question1_df' and 'question2_df' side by side.
# This combines the numerical representations of both questions into a single DataFrame.
questions_train_df = pd.concat([question1_df, question2_df], axis=1)

# Display the first few rows of the combined DataFrame 'questions_train_df'.
questions_train_df.head()


In [ ]:
question1_test_df = pd.DataFrame(ques1_test_arr)
question2_test_df = pd.DataFrame(ques2_test_arr)
ques_test_df = pd.concat([question1_test_df, question2_test_df], axis=1)
ques_test_df.head()


In [ ]:
questions_train_df.index = rest_train_data.index

In [ ]:
ques_test_df.index = rest_test_data.index

In [ ]:
df_train = pd.concat([rest_train_data,questions_train_df],axis=1)
df_train.head()

In [ ]:
df_test = pd.concat([rest_test_data,ques_test_df],axis=1)
df_test.head()

In [ ]:
X = df_train.drop('is_duplicate',axis=1)
X.columns = X.columns.astype(str)
y = df_train.is_duplicate

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=101)


In [ ]:
df_test.head()

In [ ]:
X_test = df_test.drop('is_duplicate',axis=1)
X_test.columns = X_test.columns.astype(str)
y_test = df_test.is_duplicate

In [ ]:
# Rename the columns of the training data 'X_train' to ensure they are of string data type.
X_train.columns = X.columns.astype(str)

# Extract the feature names from the DataFrame and store them in the 'features' variable.
features = X_train.columns

# Scale the training data using the previously fitted scaler ('scaler').
# This transforms the data so that it has zero mean and unit variance for each feature.
scaled_X_train = mm_scaler.fit_transform(X_train)

# Convert the scaled data back to a DataFrame while preserving the feature names.
# This allows us to work with the scaled data in a more interpretable way.
scaled_X_train = pd.DataFrame(scaled_X_train, columns=features)

# Display the first few rows of the scaled training data DataFrame.
scaled_X_train.head()


In [ ]:
scaled_X_val = mm_scaler.fit_transform(X_val)
scaled_X_val = pd.DataFrame(scaled_X_val,columns=features)
scaled_X_val.head()

In [ ]:
scaled_X_test = mm_scaler.transform(X_test)
scaled_X_test = pd.DataFrame(scaled_X_test,columns=features)
scaled_X_test.head()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Define a function to plot a confusion matrix
def plot_confusion_matrix(true_labels, predicted_labels, class_labels=None):
    # Compute the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    
    # If class_labels are not provided, create numerical labels based on unique values
    if class_labels is None:
        class_labels = np.unique(np.concatenate((true_labels, predicted_labels)))
    
    # Create a heatmap for the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)

    # Add labels, title, and axis ticks
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.xticks(np.arange(len(class_labels)) + 0.5, class_labels)
    plt.yticks(np.arange(len(class_labels)) + 0.5, class_labels)

    # Display the plot
    plt.show() 


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc

# Define a function to plot the precision-recall curve
def plot_precision_recall_curve(y_true, y_prob):
    
    # Compute precision and recall values and the area under the curve (AUC)
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    average_precision = auc(recall, precision)

    # Create a new figure for the plot
    plt.figure(figsize=(8, 6))
    
    # Plot the precision-recall curve as a step plot
    plt.step(recall, precision, color='b', alpha=0.8, where='post')
    
    # Fill the area under the curve with a transparent color
    plt.fill_between(recall, precision, alpha=0.2, color='b')

    # Add labels and set limits for the axes
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])

    # Set the title of the plot to include the Average Precision (AP) score
    plt.title(f'Precision-Recall Curve (AP={average_precision:.2f})')
    
    # Enable the grid for better readability
    plt.grid(True)
    
    # Show the plot
    plt.show()


In [ ]:
# Function to train and evaluate a  model
def train_model(model):
    # Train the model on the scaled training data and corresponding labels
    model.fit(scaled_X_train, y_train)
    
    # Calculate and print the validation accuracy
    accuracy = model.score(X_val, y_val)
    print("Validation Accuracy:", accuracy)
    
    # Make predictions on the scaled test data
    pred = model.predict(scaled_X_test)
    
    # Print a classification report to evaluate model performance
    print(classification_report(y_test, pred))
    
    # Plot a confusion matrix to visualize classification results
    plot_confusion_matrix(y_test, pred)
    
    # Plot a precision-recall curve to assess precision and recall trade-offs
    plot_precision_recall_curve(y_test, pred)


In [ ]:
# Train and evaluate a Logistic Regression model using the train_model function.
train_model(LogisticRegression())


In [ ]:
# Train and evaluate a RandomForestClassifier model using the train_model function.
train_model(RandomForestClassifier())

In [ ]:
# Train and evaluate a DecisionTreeClassifier model using the train_model function.
train_model(DecisionTreeClassifier())

In [ ]:
# Train and evaluate a LinearSVC model using the train_model function.
train_model(LinearSVC())

In [ ]:
# Train and evaluate a GradientBoostingClassifier model using the train_model function.
train_model(GradientBoostingClassifier())

In [ ]:
# Train and evaluate a AdaBoostClassifier model using the train_model function.
train_model(AdaBoostClassifier())

In [ ]:
# Define a function to create an Artificial Neural Network (ANN) model
def create_neural_network_model():
    # Create a sequential model for the ANN
    model = Sequential()
    
    # Add a dense layer with 32 units and ReLU activation function
    model.add(Dense(units=32, activation='relu'))
    
    # Add another dense layer with 64 units and ReLU activation function
    model.add(Dense(units=64, activation='relu'))
    
    # Add a third dense layer with 128 units and ReLU activation function
    model.add(Dense(units=128, activation='relu'))
    
    # Add a dropout layer with a dropout rate of 0.28
    model.add(Dropout(0.28))
    
    # Add a final dense layer with 1 unit and sigmoid activation function for binary classification
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model with binary cross-entropy loss and the Adam optimizer
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [ ]:
model = create_neural_network_model()
model

In [ ]:
# Define the EarlyStopping callback to prevent overfitting by monitoring validation accuracy.
# It will stop training if validation accuracy doesn't improve for 40 consecutive epochs.
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=40, verbose=1)

# Define the ReduceLROnPlateau callback to adjust the learning rate during training.
# It monitors validation accuracy and reduces the learning rate by a factor of 0.1 if there's no improvement for 5 consecutive epochs.
rl = ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience=5, verbose=2, min_lr=0.001, factor=0.1)

# Train the ANN model with the specified callbacks.
# It trains for 50 epochs, using a batch size of 32, and validates on the validation dataset.
# The callbacks, including EarlyStopping and ReduceLROnPlateau, are applied during training.
ann_model = model.fit(scaled_X_train,
                     y_train,
                     epochs=50,
                     batch_size=32,
                     validation_data=(scaled_X_val, y_val),
                     callbacks=[es, rl])


In [ ]:
# Create a figure for the loss plot with a specified figure size
plt.figure(figsize=(12, 8))

# Plot the training loss in red ('r') and label it as 'train loss'
plt.plot(ann_model.history['loss'], 'r', label='train loss')

# Plot the validation loss in blue ('b') and label it as 'test loss'
plt.plot(ann_model.history['val_loss'], 'b', label='test loss')

# Add labels to the x and y axes
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')

# Set the title of the plot
plt.title('Loss Graph')

# Add a legend to the plot to distinguish between training and validation loss curves
plt.legend()

# Display the plot


In [ ]:
#Referred by Kaggle Notebook
# Create a figure for the accuracy plot with a specified figure size
plt.figure(figsize=(12, 8))

# Plot the training accuracy in red ('r') and label it as 'train accuracy'
plt.plot(ann_model.history['accuracy'], 'r', label='train accuracy')

# Plot the validation accuracy in blue ('b') and label it as 'test accuracy'
plt.plot(ann_model.history['val_accuracy'], 'b', label='test accuracy')

# Add labels to the x and y axes
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')

# Set the title of the plot
plt.title('Accuracy Graph')

# Add a legend to the plot to distinguish between training and validation accuracy curves
plt.legend()

# Display the plot


In [ ]:
# Evaluate the model on the scaled test data (scaled_X_test) and corresponding labels (y_test)
loss, acc = model.evaluate(scaled_X_test, y_test)

# Print the test loss
print("Test Loss:", loss)

# Print the test accuracy
print("Test Accuracy:", acc)


In [ ]:
# Define a function to create a Convolutional Neural Network (CNN) model
def create_cnn_model(input_shape, filters, kernel_size, pool_size):
    # Create a sequential model
    model = Sequential()
    
    # Add a 1D convolutional layer with specified filters and kernel size, using ReLU activation
    model.add(Conv1D(filters, kernel_size, activation='relu', input_shape=input_shape))
    
    # Add a max pooling layer with the specified pool size
    model.add(MaxPooling1D(pool_size=pool_size))
    
    # Flatten the output of the convolutional layer
    model.add(Flatten())
    
    # Add a dense layer with 128 units and ReLU activation
    model.add(Dense(128, activation='relu'))
    
    # Add a final dense layer with 1 unit and sigmoid activation for binary classification
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model with the Adam optimizer and binary cross-entropy loss
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [ ]:
filters = 64
kernel_size = 3
pool_size = 2

In [ ]:
cnn_model = create_cnn_model(input_shape=(scaled_X_train.shape[1], 1),filters=filters, kernel_size=kernel_size, pool_size=pool_size)

In [ ]:
scaled_X_train.shape
scaled_X_train_expand = np.expand_dims(scaled_X_train, axis = -1)

In [ ]:
scaled_X_val.shape
scaled_X_val_expand = np.expand_dims(scaled_X_val, axis = -1)

In [ ]:
scaled_X_val_expand.shape

In [ ]:
es = EarlyStopping(monitor='val_accuracy',mode='max',patience=40,verbose=1)

rl = ReduceLROnPlateau(monitor='val_accuracy',mode='max',patience=5,verbose=2,min_lr=0.001,factor=0.1)

model_cnn = cnn_model.fit(scaled_X_train_expand,
         y_train,
         epochs=50,
         batch_size=32,
         validation_data=(scaled_X_val_expand,y_val),
         callbacks=[es,rl])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(model_cnn.history['loss'],'r',label='train loss')
plt.plot(model_cnn.history['val_loss'],'b',label='test loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('Loss Graph')
plt.legend()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(model_cnn.history['accuracy'],'r',label='train accuracy')
plt.plot(model_cnn.history['val_accuracy'],'b',label='test accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend()

In [ ]:
loss, acc = cnn_model.evaluate(scaled_X_test,y_test)
print("Test Loss:",loss)
print("Test Accuracy:",acc)

In [ ]:
from keras.layers import SimpleRNN, Dense
from keras.initializers import GlorotUniform

def create_rnn_model(input_shape, units):

    model = Sequential()
    model.add(SimpleRNN(units, activation='tanh',kernel_initializer = GlorotUniform(), input_shape=input_shape))
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
units = 64
input_shape=(scaled_X_train.shape[1], 1)
rnn_model = create_rnn_model(input_shape, units)

In [ ]:
scaled_X_train.shape
scaled_X_train_expand = np.expand_dims(scaled_X_train, axis = -1)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=40, verbose=1)
rl = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, verbose=2, min_lr=0.001, factor=0.1)


model_rnn = rnn_model.fit(scaled_X_train_expand,
         y_train,
         epochs=50,
         batch_size=32,
         validation_data=(scaled_X_val_expand,y_val),
         callbacks=[es,rl])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(model_rnn.history['loss'],'r',label='train loss')
plt.plot(model_rnn.history['val_loss'],'b',label='test loss')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.title('Loss Graph')
plt.legend()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(model_rnn.history['accuracy'],'r',label='train accuracy')
plt.plot(model_rnn.history['val_accuracy'],'b',label='test accuracy')
plt.xlabel('Number of Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend()

In [ ]:
loss, acc = rnn_model.evaluate(scaled_X_test,y_test)
print("Test Loss:",loss)
print("Test Accuracy:",acc)